Data to prepare:
1. Dataset-publication Interaction (Graph DP)
2. Dataset features
3. Publication features

In [20]:
PATH_data_import = "/Users/lizhouf/Documents/4_MICA/Scholarly Object Management + RecSys/DataChat/Data_raw/"
PATH_data_export = "/Users/lizhouf/Documents/4_MICA/Scholarly Object Management + RecSys/DataChat/Data_result/"

In [21]:
import pandas as pd
import numpy as np
import re
import json

In [22]:
df_studies_all = pd.read_csv(PATH_data_import+"ICPSR_datasets.csv")
df_studies_meta = pd.read_csv(PATH_data_import+"study_data_20230217.csv")
df_bib = pd.read_csv(PATH_data_import+"ICPSR_publications.csv")
df_stud_bib = pd.read_csv(PATH_data_import+"ICPSR_datasets_publications_interaction.csv")

/Users/lizhouf/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (13) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/lizhouf/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (7,13,53,54,55,56,57,58,63,64,65) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


# Preprocess datasets

In [23]:
df_studies_all.fillna("NoRecord",inplace=True)

In [24]:
df_studies_all["DESCRIPTION"]=df_studies_all["DESCRIPTION_1"]+df_studies_all["DESCRIPTION_2"]+df_studies_all["DESCRIPTION_3"]+df_studies_all["DESCRIPTION_4"]+df_studies_all["DESCRIPTION_5"]
def striphtml(data):
    p = re.compile(r'<.*?>')
    return p.sub('', str(data))
df_studies_all.DESCRIPTION = df_studies_all.DESCRIPTION.apply(lambda x: striphtml(x))

In [25]:
df_studies_all.drop(columns = ['ALTTITLE1', 'ALTTITLE2', 'ALTTITLE3', 'ALTTITLE4',
                               'ALTTITLE5', 'ALTTITLE6', 'ALTTITLE7',
                               'DESCRIPTION_1', 'DESCRIPTION_2','DESCRIPTION_3',
                               'DESCRIPTION_4', 'DESCRIPTION_5'],inplace=True)

In [26]:
df_studies_all.STUDY = df_studies_all.STUDY.astype(int).astype(int)

In [27]:
df_studies_all.ORIGRELDATE = df_studies_all.ORIGRELDATE.apply(lambda x: x[:10])

In [28]:
df_studies_all["YEAR"] = df_studies_all.ORIGRELDATE.apply(lambda x: x[:4])

In [29]:
# limit to datasets after 2017
df_studies_all = df_studies_all[df_studies_all.YEAR.isin(["2017","2018","2019","2020","2021","2022"])].reset_index(drop=True)

In [30]:
df_studies_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1642 entries, 0 to 1641
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   STUDY          1642 non-null   int64  
 1   NAME           1642 non-null   object 
 2   SERIES         1642 non-null   float64
 3   SERIES_TITLE   1642 non-null   object 
 4   OWNER          1642 non-null   object 
 5   PERMIT         1642 non-null   object 
 6   OBJECTTYPE     1642 non-null   object 
 7   FUNDINGAGENCY  1642 non-null   object 
 8   DOI            1642 non-null   object 
 9   GEO            1642 non-null   object 
 10  TERMS          1642 non-null   object 
 11  ORIGRELDATE    1642 non-null   object 
 12  DESCRIPTION    1642 non-null   object 
 13  YEAR           1642 non-null   object 
dtypes: float64(1), int64(1), object(12)
memory usage: 179.7+ KB


# augment 1: add more metadata

In [31]:
users_dict = dict(zip(df_studies_meta.STUDY,df_studies_meta[" users_to_20220822"]))
data_users_dict = dict(zip(df_studies_meta.STUDY,df_studies_meta[" datausers_to_20220822"]))
df_studies_all["users"] = [0]*len(df_studies_all)
df_studies_all["data_users"] = [0]*len(df_studies_all)
for i in range(len(df_studies_all)):
    this_study = df_studies_all.STUDY[i]
    if this_study in users_dict:
        this_num = users_dict[this_study]
        try:
            df_studies_all["users"][i] = int(this_num)
        except:
            0
    if this_study in data_users_dict:
        this_num = data_users_dict[this_study]
        try:
            df_studies_all["data_users"][i] = int(this_num)
        except:
            0


/var/folders/16/dh6sy2v17_l39d8bcf_6y7n00000gr/T/ipykernel_49032/2111330215.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_studies_all["users"][i] = int(this_num)
/var/folders/16/dh6sy2v17_l39d8bcf_6y7n00000gr/T/ipykernel_49032/2111330215.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_studies_all["data_users"][i] = int(this_num)


# augment 2: add data citing number: NOT NEEDED, REMOVE THIS!

In [32]:
df_studies_all["citations"] = [0]*len(df_studies_all)
for data_refs in df_bib.STUD_NUMS:
    try:
        data_refs_list = data_refs.split(";")
        for ref in data_refs_list:
            try: 
                df_studies_all.loc[df_studies_all.STUDY==int(ref),"citations"]+=1
            except:
                0
    except:
        0

In [33]:
df_bib.columns

Index(['REF_ID', 'REF_DOI', 'TITLE', 'AUTHORS', 'SEC_TITLE', 'ISSN',
       'STUD_NUMS', 'SERIES_NUMS', 'DATE_INPUT', 'SUBJECT', 'WHERE_FOUND',
       'STATUS', 'RIS_TYPE', 'FUNDER', 'DATE_PUB', 'YEAR_PUB', 'id', 'title',
       'altmetric_id', 'authors', 'category_for', 'concepts', 'linkout',
       'open_access', 'publisher', 'recent_citations', 'times_cited', 'type',
       'year', 'journal.id', 'journal.title', 'abstract', 'altmetric',
       'category_sdg', 'category_uoa', 'reference_ids', 'referenced_pubs',
       'research_orgs', 'researchers', 'category_hra', 'category_bra',
       'field_citation_ratio', 'relative_citation_ratio', 'funders',
       'supporting_grant_ids', 'STUDY'],
      dtype='object')

# augment 3: add publications details

In [42]:
def remove_duplicate_dict(l):
    seen = set()
    new_l = []
    for d in l:
        t = d["name"]
        if t not in seen:
            seen.add(t)
            new_l.append(d)
    return new_l

In [43]:
 remove_duplicate_dict([{"name":"a","pubRefCount":1},{"name":"a","pubRefCount":3},{"name":"b","pubRefCount":3}])

[{'name': 'a', 'pubRefCount': 1}, {'name': 'b', 'pubRefCount': 3}]

In [44]:
# name, url, citations
publication_list = []

for i in range(len(df_studies_all)):
    if i%100==0:
        print(i)
    this_study = df_studies_all.STUDY[i]
    this_pubs_list = []
    
    for j in range(len(df_bib)):
        data_refs = df_bib.STUD_NUMS[j]
        if type(data_refs)==str:
            data_refs_list = [int(ref) for ref in data_refs.split(";")]
            if this_study in data_refs_list:
                ref_count = df_bib.times_cited[j]
                if np.isnan(ref_count):
                    ref_count = 0
                this_pubs_list.append({"name": df_bib.TITLE[j],
                                       "pubRefCount": ref_count})

    publication_list.append(remove_duplicate_dict(this_pubs_list))

df_studies_all["publications"] = publication_list

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600


# export results

In [45]:
data_results = list()
for idx, row in df_studies_all.iterrows():
    data_results.append({"id":row["STUDY"],
                         "name":row["NAME"],
                         "date":row["ORIGRELDATE"],
                         "url":"https://doi.org/"+row["DOI"], 
                         "series":row["SERIES_TITLE"],
                         "owner":row["OWNER"],
                         "users":row["users"],
                         "dataUsers":row["data_users"],
                         "dataReferences":row["citations"],
                         "funders":[{"name":funder} for funder in row["FUNDINGAGENCY"].split(".")], 
                         "locations":[{"name":location} for location in row["GEO"].split(";")],
                         "terms":[{"name":term} for term in row['TERMS'].split(";")],
                         "publications":row["publications"]
                        })

In [46]:
data_results[:3]

[{'id': 25541,
  'name': 'Channels of Interstate Risk Sharing, United States, 1963-2000',
  'date': '2018-06-18',
  'url': 'https://doi.org/10.3886/ICPSR25541.v1 ',
  'series': 'NoRecord',
  'owner': 'ICPSR',
  'users': 0,
  'dataUsers': 0,
  'dataReferences': 1,
  'funders': [{'name': 'National Science Foundation'}],
  'locations': [{'name': 'United States'}],
  'terms': [{'name': 'capitalism'},
   {'name': ' consumer expenditures'},
   {'name': ' credit'},
   {'name': ' economic activity'},
   {'name': ' economic conditions'},
   {'name': ' economic forecasting'},
   {'name': ' economic indicators'},
   {'name': ' economic sectors'},
   {'name': ' economic systems'},
   {'name': ' grants'},
   {'name': ' Gross Domestic Product'},
   {'name': ' Gross National Product'},
   {'name': ' market economy'},
   {'name': ' risk'},
   {'name': ' taxes'}],
  'publications': [{'name': 'Channels of interstate risk sharing: United States 1963--1990',
    'pubRefCount': 457.0}]},
 {'id': 26343,
  '

In [47]:
len(data_results)

1642

In [48]:
with open(PATH_data_export+"data_output_17_22.json", "w") as fout:
    json.dump(data_results, fout)